📌 1. Imports

In [ ]:
# Regular expressions for parsing input lines
import re

# Dictionary with default values for nested transitions
from collections import defaultdict

# For drawing the DFA using Graphviz
import graphviz

# For uploading files in Google Colab
from google.colab import files

📌 2. DFA Class Definition and Initialization

In [ ]:
class DFA:
    def __init__(self, alphabet, states, initial, finals, transitions):
        # Basic DFA components
        self.alphabet = alphabet
        self.states = states
        self.initial = initial
        self.finals = finals
        self.transitions = transitions


📌 3. Loading DFA from a .txt File

In [ ]:
    @staticmethod
    def from_txt(file_path):
        # Load and parse the DFA definition from a formatted text file
        with open(file_path, 'r') as f:
            lines = [line.strip() for line in f if line.strip() and not line.startswith('#')]

        alphabet = states = finals = transitions = None
        initial = None
        trans = []

        for line in lines:
            if line.startswith('alfabeto:'):
                alphabet = line.split(':')[1].split(',')
            elif line.startswith('estados:'):
                states = line.split(':')[1].split(',')
            elif line.startswith('inicial:'):
                initial = line.split(':')[1]
            elif line.startswith('finais:'):
                finals = line.split(':')[1].split(',')
            elif re.match(r'^\w+,\w+,\w+$', line):
                trans.append(tuple(line.split(',')))

        if not all([alphabet, states, initial, finals, trans]):
            raise ValueError("Input file is incomplete or malformed.")

        transitions = defaultdict(dict)
        for origin, dest, symbol in trans:
            if symbol in transitions[origin]:
                raise ValueError(f"Nondeterministic transition from {origin} on symbol {symbol}.")
            transitions[origin][symbol] = dest

        dfa = DFA(alphabet, states, initial, finals, transitions)
        dfa.validate()
        return dfa


📌 4. Validation Method

In [ ]:
    def validate(self):
        # Ensure all states and symbols are valid
        if self.initial not in self.states:
            raise ValueError("Initial state is not in the list of states.")
        for f in self.finals:
            if f not in self.states:
                raise ValueError(f"Final state {f} is not valid.")
        for origin, trans in self.transitions.items():
            if origin not in self.states:
                raise ValueError(f"Invalid origin state: {origin}")
            for symbol, dest in trans.items():
                if symbol not in self.alphabet:
                    raise ValueError(f"Invalid symbol in transition: {symbol}")
                if dest not in self.states:
                    raise ValueError(f"Invalid destination state: {dest}")


📌 5. Remove Unreachable States

In [ ]:
    def remove_unreachable_states(self):
        reachable = set()

        def visit(state):
            if state in reachable:
                return
            reachable.add(state)
            for symbol in self.alphabet:
                dest = self.transitions.get(state, {}).get(symbol)
                if dest and dest not in reachable:
                    visit(dest)

        visit(self.initial)

        self.states = [s for s in self.states if s in reachable]
        self.finals = [f for f in self.finals if f in reachable]
        self.transitions = defaultdict(dict, {
            s: {sym: d for sym, d in trans.items() if d in reachable}
            for s, trans in self.transitions.items() if s in reachable
        })


📌 6. DFA Minimization with Table Filling

In [ ]:
    def minimize(self):
        print("\nStarting minimization using the Table Filling method...\n")
        print("Step 1: Removing unreachable states...")
        self.remove_unreachable_states()
        print(f"Reachable states: {self.states}\n")

        n = len(self.states)
        index = {state: i for i, state in enumerate(self.states)}
        rev_index = {i: state for i, state in enumerate(self.states)}
        table = [[False for _ in range(n)] for _ in range(n)]

        print("Step 2: Marking (final, non-final) state pairs as distinguishable...")
        for i in range(n):
            for j in range(i):
                s1, s2 = rev_index[i], rev_index[j]
                if (s1 in self.finals) != (s2 in self.finals):
                    table[i][j] = True
                    print(f"  Marked as distinguishable: ({s1}, {s2})")
        print()

        print("Step 3: Propagating distinguishability...")
        changed = True
        while changed:
            changed = False
            for i in range(n):
                for j in range(i):
                    if table[i][j]:
                        continue
                    for symbol in self.alphabet:
                        t1 = self.transitions.get(rev_index[i], {}).get(symbol)
                        t2 = self.transitions.get(rev_index[j], {}).get(symbol)
                        if not t1 or not t2:
                            continue
                        i1, i2 = index[t1], index[t2]
                        if table[max(i1, i2)][min(i1, i2)]:
                            table[i][j] = True
                            changed = True
                            print(f"  ({rev_index[i]}, {rev_index[j]}) distinguished by {symbol} → ({t1}, {t2})")
                            break
        print()

        print("Step 4: Grouping equivalent states...")
        groups = []
        marked = set()
        for i in range(n):
            if self.states[i] in marked:
                continue
            group = [self.states[i]]
            for j in range(i):
                if not table[i][j]:
                    group.append(self.states[j])
                    marked.add(self.states[j])
            marked.add(self.states[i])
            groups.append(group)

        for idx, group in enumerate(groups, 1):
            print(f"  Group {idx}: {group}")
        print()

        print("Step 5: Creating new minimized DFA...")
        name_map = {}
        new_states = []
        for group in groups:
            name = ', '.join(sorted(group))
            new_states.append(name)
            for state in group:
                name_map[state] = name

        new_initial = name_map[self.initial]
        new_finals = list(set(name_map[f] for f in self.finals))

        print(f"  New states: {new_states}")
        print(f"  New initial: {new_initial}")
        print(f"  New finals: {new_finals}\n")

        new_transitions = defaultdict(dict)
        for group in groups:
            rep = group[0]
            new_state = name_map[rep]
            for symbol in self.alphabet:
                dest = self.transitions.get(rep, {}).get(symbol)
                if dest:
                    new_transitions[new_state][symbol] = name_map[dest]
                    print(f"  {new_state} --{symbol}--> {name_map[dest]}")
        print("\nMinimization complete!\n")

        return DFA(self.alphabet, new_states, new_initial, new_finals, new_transitions)


📌 7. Drawing the DFA with Graphviz

In [ ]:
    def draw(self, filename='dfa_minimized'):
        dot = graphviz.Digraph()

        used = set(self.transitions.keys())
        for trans in self.transitions.values():
            used.update(trans.values())

        for state in used:
            shape = 'doublecircle' if state in self.finals else 'circle'
            dot.node(state, shape=shape)

        dot.node('', shape='none')
        dot.edge('', self.initial)

        grouped = defaultdict(list)
        for origin, trans in self.transitions.items():
            for symbol, dest in trans.items():
                grouped[(origin, dest)].append(symbol)

        for (origin, dest), symbols in grouped.items():
            label = '/'.join(sorted(symbols))
            dot.edge(origin, dest, label=label)

        dot.render(filename, format='png', cleanup=True)
        print(f"Diagram saved as {filename}.png")
